# Fine-tuning Gemma using Ludwig on GST-FAQs dataset


🙌 Welcome to the hands-on tutorial dedicated to exploring the cutting-edge capabilities of [Ludwig](https://ludwig.ai/latest/) 0.8, for building an Question Answering model for FAQs (Frequently Asked Questions) on GST (Goods and Services Tax) in India.

Ludwig, an open-source package has been used here to train machine learning models in Encoder-Combination-Decoder (ECD) mode as well as in fine-tuning LLMs via Instruction Tuning mode, through declarative config files.

A bit more info about GST:  GST is a single tax-structure that replaces a multitude of taxes that were there before in India, such as the service tax, central excise duty, VAT, and more. It's the all-in-one tax solution that streamlines the entire tax process in India. This transition from mutlitude-tax system to a single-tax system, raises lots of queries. These queries, along with their answers are avaiable as FAQs. Building a ML model or a fine-tuned LLM would surely help build a chatbot like application on top.

👉👉 Step-by-step explanation of the solution is available [here](https://medium.com/analytics-vidhya/how-to-fine-tune-llms-without-coding-41cf8d4b5d23).

## Installation 🧰

Needs HuggingFace API Token, access approval to Gemma–7b-it, and a GPU with a minimum of 12 GiB of VRAM. Here in this notebook, T4 GPU is being used.

In [1]:
!pip uninstall -y tensorflow --quiet
!pip install Cython # do this before installing torch which is inside ludwig, to avoid "_C" error
!pip install ludwig
!pip install ludwig[llm]
!pip install accelerate
from accelerate.utils import write_basic_config; write_basic_config(mixed_precision='fp16')
!pip install -i https://pypi.org/simple/ bitsandbytes  # latest
# !pip install bitsandbytes==0.41.3 --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui # overriding 0.40.2 which comes with Ludwig
#You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting require

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 18.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple/


Enable text wrapping so we don't have to scroll horizontally and create a

---

function to flush CUDA cache.

In [2]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    model = None
    torch.cuda.empty_cache()

Sometime error comes as ```NameError: name '_C' is not defined``, follow https://github.com/pytorch/pytorch/issues/1633 for the solution

-> **Setup Your HuggingFace Token** 🤗

We'll be using  Llama-2, which a model released by Meta. However, the model is not openly-accessible and requires requesting for access (assigned to your HuggingFace token).

Obtain a [HuggingFace API Token](https://huggingface.co/settings/tokens) and request access to [gemma-7b-it](https://huggingface.co/google/gemma-7b-it) before proceeding. You may need to signup on HuggingFace if you don't aleady have an account: https://huggingface.co/join

In [3]:
import getpass
# import locale; locale.getpreferredencoding = lambda: "utf-8"
import logging
import os
import torch
import yaml

from ludwig.api import LudwigModel


os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
Token:··········


In [4]:
# os.environ["HF_TOKEN"] = getpass.getpass("Token:")
# assert os.environ["HF_TOKEN"] #give same as HUGGING_FACE_HUB_TOKEN

## Configurations


Defining config for Instruction Fine Tuning using Mistral 7B model. It is based on [this](https://predibase.com/blog/fine-tuning-mistral-7b-on-a-single-gpu-with-ludwig) tutorial. Prompt has been changed.

In [5]:
instruction_tuning_yaml = yaml.safe_load("""
model_type: llm
base_model: google/gemma-7b-it

quantization:
 bits: 4

adapter:
 type: lora

prompt:
  template: |
    ### Instruction:
    You are a taxation expert on Goods and Services Tax used in India.
    Take the Input given below which is a Question. Give Answer for it as a Response.

    ### Input:
    {Question}

    ### Response:

input_features:
 - name: Question
   type: text
   preprocessing:
      max_sequence_length: 1024

output_features:
 - name: Answer
   type: text
   preprocessing:
      max_sequence_length: 384

trainer:
  type: finetune
  epochs: 5
  batch_size: 1
  eval_batch_size: 2
  gradient_accumulation_steps: 16  # effective batch size = batch size * gradient_accumulation_steps
  learning_rate: 2.0e-4
  enable_gradient_checkpointing: true
  learning_rate_scheduler:
    decay: cosine
    warmup_fraction: 0.03
    reduce_on_plateau: 0

generation:
  temperature: 0.1
  max_new_tokens: 512

backend:
 type: local
""")

## Dataset
Data in the form of csv is made avilable at the Github location [here](https://raw.githubusercontent.com/yogeshhk/Sarvadnya/master/src/ludwig/data/cbic-gst_gov_in_fgaq.csv). `wget` it ones from the location given below. Keep it in `data` folder, then comment this cell for further executions.

In [6]:
# !pip install wget
# import wget

# # Replace the URL with the raw URL of the file on GitHub
# url = "https://raw.githubusercontent.com/yogeshhk/Sarvadnya/master/src/ludwig/data/cbic-gst_gov_in_fgaq.csv"

# # Download the file
# wget.download(url, 'cbic-gst_gov_in_fgaq.csv')

-> Needs permission. Change to drive location below to where the csv file needed for the notebook resides.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ImpDocs/Work/AICoach/Notebooks')

Mounted at /content/drive


In [8]:
from google.colab import data_table; data_table.enable_dataframe_formatter()
import numpy as np; np.random.seed(123)
import pandas as pd

Change to drive location below to where the csv file needed for the notebook resides.

In [9]:
df = pd.read_csv('/content/drive/MyDrive/ImpDocs/Work/AICoach/Notebooks/data/cbic-gst_gov_in_fgaq.csv', encoding='cp1252')
df.head()


,Question,Answer
0,Does aggregate turnover include value of inwar...,Refer Section 2(6) of CGST Act. Aggregate turn...
1,What if the dealer migrated with wrong PAN as ...,New registration would be required as partners...
2,A taxable person’s business is in many states....,He is liable to register if the aggregate turn...
3,Can we use provisional GSTIN or do we get new ...,Provisional GSTIN (PID) should be converted in...
4,Whether trader of country liquor is required t...,If the person is involved in 100% supply of go...


A crucial step in our journey involves the compilation of a dataset that mirrors the real-world questions taxpayers grapple with. So, this dataset is a Question Answering dataset. Each row in the dataset consists of an:
- `Question` that describes a query
- `Answer` that describes the correspondng answer

## Running Ludwig: Training

The model's declarative nature allows us to clearly define the architecture, making the training process transparent and insightful.

Instantiation of `LudwigModel` with fine-tuning config `instruction_tuning_yaml`. Training it on GST csv based dataframe.

In [ ]:
model_instruction_tuning = LudwigModel(config=instruction_tuning_yaml,  logging_level=logging.INFO)
results_instruction_tuning = model_instruction_tuning.train(dataset=df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/drive/MyDrive/ImpDocs/Work/AICoach/Notebooks/results/api_experiment_run_4      │
├──────────────────┼─────────────────────────────────────────────────────────────────

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of google/gemma-7b-it tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:ludwig.features.text_feature:Max length of feature 'None': 104 (without start and stop symbols)
INFO:ludwig.features.text_feature:Max sequence length is 104 for feature 'None'
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of google/gemma-7b-it tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:ludwig.features.text_feature:Max length of feature 'Answer': 95 (without start and stop symbols)
INFO:ludwig.features.text_feature:Max sequence length is 95 for feature 'Answer'
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of google/gemma-7b-it tokenizer
Asking to truncate to max_length but no maximum length is provided an

Testing or inferencing dataset has just a couple of questions for which answers are seeked.

In [ ]:
import pandas as pd
test_df = pd.DataFrame([
    {
        "Question": "If I am not an existing taxpayer and wish to newly register under GST, when can I do so?"
    },
    {
        "Question": "Does aggregate turnover include value of inward supplies received on which RCM is payable?"
    },
])


## Runnuing Ludwig: Inferencing

With Ludwig's training complete, the explorers put the model to the test. They fed it a set of questions related to GST, eager to witness the declarative AI framework in action.

**Predictions on fine-tuned model**

In [ ]:
predictions_instruction_tuning_df, output_directory = model_instruction_tuning.predict(dataset=test_df)
print(predictions_instruction_tuning_df["Answer_response"].tolist())

The answres are `[['nobody can be registered under gst unless he is liable to be registered under section 22 of the cgst act, 2017 read with section 2(6) of the sgst act, 2017.'], ['nobody is liable to pay rcm on inward supplies.']]`

These are reasonably ok, but both answers starting with `nobody` seems to be a little odd. There could be many reasons, quality of LLM, training paramater, and above all, need far bigger bigger dataset for fine-tuning.

## **Observations** 🔎

Fine-tunined model seems to have given decent results. Ludwig's declarative approach provides a clear and concise methodology for building machine learning models, making it an invaluable tool for unraveling the mysteries of complex domains. It becomes extreamly easy to change between these approaches, change base LLMs etc.

# **Resources** 🧺
- How to Efficiently Fine-Tune Gemma-7B with Open-Source Ludwig https://predibase.com/blog/how-to-efficiently-fine-tune-gemma-7b-with-open-source-ludwig
- Fine-tuning Mistral 7B on a Single GPU with Ludwig https://predibase.com/blog/fine-tuning-mistral-7b-on-a-single-gpu-with-ludwig
- Efficient Fine-Tuning for Llama-v2-7b on a Single GPU https://www.youtube.com/watch?v=g68qlo9Izf0
- If you're new to LLMs, check out this webinar where Daliana Liu discusses the 10 things to know about LLMs: https://www.youtube.com/watch?v=fezMHMk7u5o&t=2027s&ab_channel=Predibase
- Ludwig 0.8 Release Blogpost for the full set of new features: https://predibase.com/blog/ludwig-v0-8-open-source-toolkit-to-build-and-fine-tune-custom-llms-on-your-data
- Ludwig Documentation: https://ludwig.ai/latest/